In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path = [".."] + sys.path

In [ ]:
import src.optimal_assignment as oa
import src.basic_assignment as ba

## Get distance between zipcodes

In [ ]:
observers = ba.get_observer_dataset()
precinct = ba.get_precinct_dataset()
precinct, observers = ba.run_ordered_assignment(precinct, observers)

In [ ]:
mask = (precinct["outside_am_legal"] == True) & (
    precinct["outside_am_observer"] != precinct["outside_pm_observer"]
)
precinct_subset = precinct[mask]

In [ ]:
column_to_optimise = "outside_am_observer"

merged_df = precinct_subset.merge(
    observers[["name", "post_code"]], left_on=column_to_optimise, right_on="name"
)
precinct_list = merged_df[["Polling Place Name", "Zip"]]
observer_list = merged_df[[column_to_optimise, "post_code"]]
distance = np.abs(
    precinct_list.Zip.values[np.newaxis, :]
    - observer_list.post_code.values[:, np.newaxis]
)
distance_df = pd.DataFrame(
    distance,
    index=observer_list[column_to_optimise],
    columns=precinct_list["Polling Place Name"],
)

merged_df["current_distance"] = np.abs(merged_df["Zip"] - merged_df["post_code"])

In [ ]:
matched_set = oa.get_matched_sets(distance_df, merged_df, column_to_optimise, verbose=False)

In [ ]:
df = pd.DataFrame([matched_set], index=["pollingstation"]).T.reset_index()
df.columns = [column_to_optimise, "pollingstation"]
out = precinct[["Pct", "Polling Place Name"]].merge(
    df, left_on="Polling Place Name", right_on="pollingstation"
).drop("pollingstation", axis=1)

In [ ]:
precinct.loc[mask, "outside_am_observer"] = out['outside_am_observer'].values